In [1]:
"""
评估生成的图片和真实图片的相似度
"""

from PIL import Image  # type: ignore
import os
import random
from captcha.image import ImageCaptcha, random_color  # type: ignore
from PIL.ImageFilter import SMOOTH


class NewImageCaptcha(ImageCaptcha):
    def __init__(self, width: int = 160, height: int = 60, fonts: list[str] | None = None, font_sizes: tuple[int, ...] | None = None):
        super().__init__(width, height, fonts, font_sizes)
        self.character_warp_dx = (0.01, 0.02)
        self.character_warp_dy = (0.01, 0.02)
        self.word_space_probability = 0
        self.word_offset_dx = 0.1
        self.character_rotate = (-15, 15)

    def generate_image(self, chars: str):
        """Generate the image of the given characters.

        :param chars: text to be generated.
        """
        background = random_color(170, 220)
        color = random_color(10, 200, random.randint(220, 255))
        im = self.create_captcha_image(chars, color, background)
        im = im.filter(SMOOTH)
        return im


def generate_captcha(count, width=163, height=58):
    img_list = []
    # 生成验证码图片
    captcha = NewImageCaptcha(width=width, height=height,
                              fonts=[
                                  '../fonts/Open_Sans/static/OpenSans-Light.ttf'],
                              font_sizes=(18, 20, 22))

    for i in range(count):
        # k 表示验证码位数
        chars = ''.join(map(str, random.choices(
            '0123456789abcdefghijklmnopqrstuvwxyz', k=random.choice(range(4, 7)))))
        img = captcha.generate_image(chars)
        img_list.append(img)
    return img_list


def get_img_path_list(img_dir, limit):
    img_path_list = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            file_extension = os.path.splitext(file)[1]
            if file_extension in ['.jpg', '.jpeg', '.png']:
                img_path_list.append(os.path.join(root, file))

    return random.choices(img_path_list, k=limit)


def read_img(img_path):
    img = Image.open(img_path)
    return img


count = 4
# generate_img_path_list = get_img_path_list(
#     '../data/动态 4-6 位/数字 0-9a-z/train', count)
real_img_path_list = get_img_path_list(
    '../data/train-3363-stable-new/test', count)

# 根据上述 4 张图片路径, 合并成一张图片
# generate_img_list = [
#     read_img(img_path) for img_path in generate_img_path_list
# ]
generate_img_list = generate_captcha(count)
real_img_list = [
    read_img(img_path) for img_path in real_img_path_list
]

width, height = generate_img_list[0].size
# 生成图片
generate_img = Image.new('RGB', (width * count, height * 2))
for i in range(count):
    generate_img.paste(generate_img_list[i], (width * i, 0))
    generate_img.paste(real_img_list[i], (width * i, height))

generate_img.show()